In [8]:
#imports and setup
import numpy as np
import cv2 as cv
import os
from keras.models import load_model


In [15]:
video_path = "videos"
full_path = os.path.abspath(video_path)

video_lst = []
v_dir = os.listdir(full_path)
for name in v_dir:
    video_lst.append(full_path + '/' + name)
print(video_lst[5])

C:\Users\bulga\PycharmProjects\ComputerVision\videos/video_005.mp4


In [3]:
import math


class EuclideanDistTracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0

    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 25:
                    self.center_points[id] = (cx, cy)
                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

In [4]:
import cv2

cap = cv.VideoCapture(video_lst[24])
# Open the video file
video = cap
# Set the frame rate


# Read the video frame by frame
success, frame = video.read()
frame_number = 1
path = 'C:/Users/bulga/PycharmProjects/ComputerVision/Training Data/45'
while success:
    # Save the current frame as an image
    cv2.imwrite(os.path.join(path, '45{}.jpg'.format(frame_number)), frame)

    # Read the next frame
    success, frame = video.read()
    frame_number += 1

# Release the video file
video.release()
print(frame_number)

1753


In [1]:
video = video_lst[2]

kernel = np.ones((7, 7))  #kerne for dilation
kernel_blur = (3, 3)
kernel_morph = cv.getStructuringElement(cv.MORPH_ELLIPSE, (7, 7))  #kernel for morphology
THRESHOLD = 20  #when to detect difference
VALUE = 255  #which value to assign to difference

#threshold to calculate
threshold_method = cv.THRESH_BINARY
tracker = EuclideanDistTracker()


# Compute the frame difference
def frame_diff(prev_frame, cur_frame, next_frame):
    diff_frames1 = cv.absdiff(next_frame, cur_frame)
    # Absolute difference between current frame and previous frame
    diff_frames2 = cv.absdiff(cur_frame, prev_frame)
    # Return the result of bitwise 'AND' between the above two resultant images
    #gives better result than simple substraction
    return cv.bitwise_and(diff_frames1, diff_frames2)


def get_frame(cap):
    ret, frame = cap.read()
    # Resize the image not needed
    '''frame = cv.resize(frame, None, fx=scaling_factor,
        fy=scaling_factor, interpolation=cv.INTER_AREA)'''
    return frame


#finds and draws contours over pixels if greater than threshold
def construct_contours(frame, contour_threshold):
    frame = cv.dilate(frame, (1, 1), iterations=1)
    contours, hierarchy = cv.findContours(frame, cv.RETR_TREE,
                                          cv.CHAIN_APPROX_SIMPLE)
    '''cv.drawContours(frame_th, contours=contours, contourIdx=-1,
                     color=(0, 255, 0), thickness=2, lineType=cv.LINE_AA)'''
    detections = []
    for contour in contours:
        if cv.contourArea(contour) < contour_threshold:
            continue
        (x, y, w, h) = cv.boundingRect(contour)
        #cv.rectangle(frame, pt1=(x, y), pt2=(x + w, y + h), color=(0, 255, 0), thickness=2)
        detections.append([x, y, w, h])

    return detections


cap = cv.VideoCapture(video)
prev_frame = get_frame(cap)
cur_frame = get_frame(cap)
next_frame = get_frame(cap)

model = load_model('multiclass_recognition.h5')


#applying blur will possibly extend with other option for preperation
def prepare_frames(frames, kernel):
    result = []
    for frame in frames:
        result.append(cv.GaussianBlur(frame, kernel, 0))
    return result[0], result[1], result[2]

frame_number=0

# Iterating over all frames and applying difference and dilation
while True:
    frames = [prev_frame, cur_frame, next_frame]

    frame_number += 1

    # check if we need to make a prediction
    if frame_number % 30 == 0:
        img = cur_frame
        res = cv2.resize(img, dsize=(256, 256))
        x = np.asarray(res)

        # reshape the array
        x = x.reshape((1,) + x.shape)

        # make a prediction on the image
        prediction = model.predict(x)
        thr = [20, 25, 30, 45]
        class_index = np.argmax(prediction)


        THRESHOLD = thr[class_index]
        print(thr[class_index])
    real_frame = cur_frame
    #applying preprocessing
    prev_frame, cur_frame, next_frame = prepare_frames(frames, kernel_blur)
    #cv.imshow("cur_frame", cur_frame)
    #calculating difference
    frame_difference = frame_diff(prev_frame, cur_frame, next_frame)
    #cv.imshow("Difference", frame_difference)
    #applying dilation
    frame_difference = cv.dilate(frame_difference, kernel)
    #cv.imshow("dilate", frame_difference)
    #applying morphological noise reduction
    frame_difference = cv.morphologyEx(frame_difference, cv.MORPH_OPEN, kernel_morph)
    #cv.imshow("morphologyEx", frame_difference)

    #converting into greyscale for contour finding
    frame_difference = cv.cvtColor(frame_difference, cv.COLOR_BGR2GRAY)
    #cv.imshow("cvtColor", frame_difference)
    #applying thresholds
    ret, frame_th = cv.threshold(frame_difference, THRESHOLD, VALUE, threshold_method)
    #cv.imshow("threshold", frame_difference)
    #finding and drawing contours on given frame with given threshold
    detections = construct_contours(frame_th, 30)

    #cv.imshow("Difference", frame_difference)
    cv.imshow("After Threshold", frame_th)

    boxes_ids = tracker.update(detections)
    for box_id in boxes_ids:
        x, y, w, h, id = box_id
        cv.putText(real_frame, str(id), (x, y - 15), cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        cv.rectangle(real_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv.imshow('frame', real_frame)

    # Update the variables
    prev_frame = cur_frame
    cur_frame = next_frame
    next_frame = get_frame(cap)

    if cv.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

NameError: name 'video_lst' is not defined